## **The Training Data**
<p>The Landslide4Sense data consists of the training, validation, and test sets containing 3799, 245, and 800 image patches, respectively. Each image patch is a composite of 14 bands that include:

> **Multispectral data from Sentinel-2 : B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, B12**.
>
> **Slope data from ALOS PALSAR: B13**.
>
> **Digital elevation model (DEM) from ALOS PALSAR: B14**.

All bands in the competition dataset are resized to the resolution of ~10m per pixel. The image patches have the size of 128 x 128 pixels and are labeled pixel-wise.</p>

## **The Testing Data**
<p>Including UAV data of Van Yen and Mu Cang Chai districts:
    
> **High resolution ~ 0.17m**.
>
> **3 bands - Red, Green, Blue**.</p>

# **Training Data and Validation**

In [1]:
# set environment variables
import os
from utils import *

%set_env LOCAL_TRAINDATA_DIR=Data/TrainData
%set_env LOCAL_VALIDDATA_DIR=Data/ValidData
%set_env LOCAL_PARENT_DIR = Input
%set_env LOCAL_PREDICT_SET_DIR = Predict_Set

# set paths for training images, masks, and valid images
image_dir=os.path.join(os.getenv('LOCAL_TRAINDATA_DIR'), 'img')
mask_dir=os.path.join(os.getenv('LOCAL_TRAINDATA_DIR'), 'mask')
val_dir = os.path.join(os.getenv('LOCAL_VALIDDATA_DIR'), 'img')
pred_dir = os.path.join(os.getenv('LOCAL_PREDICT_SET_DIR'),'')
parent_dir = os.path.join(os.getenv('LOCAL_PARENT_DIR'), '')

env: LOCAL_TRAINDATA_DIR=Data/TrainData
env: LOCAL_VALIDDATA_DIR=Data/ValidData
env: LOCAL_PARENT_DIR=Input
env: LOCAL_PREDICT_SET_DIR=Predict_Set


### **#5 - Generate tensor input**

In [2]:
image, label, inputs = model_fetch.generate_tensor_input(parent_dir, 'image_patch128.npy', 'label_patch128.npy', 2)

Create tensor input done in 4.079807758331299 seconds


### **#6 - Train-Test split**

In [3]:
train_image, train_label, test_image, test_label = model_fetch.train_test_split(image, label, train_size=0.6, test_size=0.4)

Dims of train_image: (9117, 128, 128, 3)
Dims of train_label: (9117, 128, 128, 2)
Dims of test_image: (6079, 128, 128, 3)
Dims of test_label: (6079, 128, 128, 2)


### **#7 - Model generation**

In [4]:
prepared_model = model_fetch.model_generation(inputs, 2)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 8)  224         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 8)  32         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

### **#8 - Trainining**

In [5]:
pretrained_model, model_history = model_fetch.train_model(prepared_model, train_image, train_label, test_image, test_label, epochs=100, batch_size=30, learning_rate=0.001, rho=0.9, momentum=0.0, metrics=['accuracy'])

Epoch 1/100
304/304 [==============================] - ETA: 0s - loss: 0.0697 - accuracy: 0.9799
Epoch 1: saving model to Output/Model/Checkpoints\cp.ckpt
304/304 [==============================] - 3330s 11s/step - loss: 0.0697 - accuracy: 0.9799 - val_loss: 0.1040 - val_accuracy: 0.9808
Epoch 2/100
304/304 [==============================] - ETA: 0s - loss: 0.0493 - accuracy: 0.9800
Epoch 2: saving model to Output/Model/Checkpoints\cp.ckpt
304/304 [==============================] - 3323s 11s/step - loss: 0.0493 - accuracy: 0.9800 - val_loss: 0.0634 - val_accuracy: 0.9808
Epoch 3/100
304/304 [==============================] - ETA: 0s - loss: 0.0474 - accuracy: 0.9800
Epoch 3: saving model to Output/Model/Checkpoints\cp.ckpt
304/304 [==============================] - 3324s 11s/step - loss: 0.0474 - accuracy: 0.9800 - val_loss: 0.0442 - val_accuracy: 0.9808
Epoch 4/100
304/304 [==============================] - ETA: 0s - loss: 0.0457 - accuracy: 0.9801
Epoch 4: saving model to Output/Mode

### **#9 - Predicting**

In [7]:
# recall model
valid, predict = model_fetch._predict_(pretrained_model, predict_set=os.path.join(parent_dir, 'valid_patch128.npy'), batch_size=30)

Predict dataset loading...
Predicting...
9/9 [==============================] - 19s 2s/step
Prediction done in 20.143659353256226 seconds
